In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

def set_axis_style(ax, labels):
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)

In [4]:
folder_location = '../results/toxic_measures/'

original = json.load(open(f'{folder_location}original.json'))

opt_first_line = json.load(open(f'{folder_location}opt_first_line.json'))
opt_256_tokens = json.load(open(f'{folder_location}opt_256_tokens.json'))
opt_512_tokens = json.load(open(f'{folder_location}opt_512_tokens.json'))

llama_first_line = json.load(open(f'{folder_location}llama_first_line.json'))
llama_256_tokens = json.load(open(f'{folder_location}llama_256_tokens.json'))
llama_512_tokens = json.load(open(f'{folder_location}llama_512_tokens.json'))

alpaca_1 = json.load(open(f'{folder_location}alpaca_1.json'))
alpaca_3 = json.load(open(f'{folder_location}alpaca_3.json'))

alpaca_free_2 = json.load(open(f'{folder_location}alpaca_free_2.json'))
alpaca_free_4 = json.load(open(f'{folder_location}alpaca_free_4.json'))

def convert_to_single(dic):
    vals = {
        'toxic': [],
        'severe_toxic': [],
        'obscene': [],
        'threat': [],
        'insult': [],
        'identity_hate': [],
    }
    
    for e1 in dic:
        for e2 in e1:
            vals[e2['label']].append(e2['score'])
    return vals

original = convert_to_single(original)

opt_first_line = convert_to_single(opt_first_line)
opt_256_tokens = convert_to_single(opt_256_tokens)
opt_512_tokens = convert_to_single(opt_512_tokens)

llama_first_line = convert_to_single(llama_first_line)
llama_256_tokens = convert_to_single(llama_256_tokens)
llama_512_tokens = convert_to_single(llama_512_tokens)

alpaca_1 = convert_to_single(alpaca_1)
alpaca_3 = convert_to_single(alpaca_3)

alpaca_free_2 = convert_to_single(alpaca_free_2)
alpaca_free_4 = convert_to_single(alpaca_free_4)

In [78]:
def get_toxic_measure(label='toxic'):
    print(label)
    print('....')
    bin_labels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    # original
    counts, _ = np.histogram(original[label], bins=bin_labels)
    print(f'Real Stories: {counts}')

    counts, _ = np.histogram(opt_first_line[label], bins=bin_labels)
    print(f'OPT-Line: {counts}')
    counts, _ = np.histogram(opt_256_tokens[label], bins=bin_labels)
    print(f'OPT-256: {counts}')
    counts, _ = np.histogram(opt_512_tokens[label], bins=bin_labels)
    print(f'OPT-512: {counts}')

    counts, _ = np.histogram(llama_first_line[label], bins=bin_labels)
    print(f'LLaMA-Line: {counts}')
    counts, _ = np.histogram(llama_256_tokens[label], bins=bin_labels)
    print(f'LLaMA-256: {counts}')
    counts, _ = np.histogram(llama_512_tokens[label], bins=bin_labels)
    print(f'LLaMA-512: {counts}')

    counts, _ = np.histogram(alpaca_1[label], bins=bin_labels)
    print(f'Alpaca: T1: {counts}')
    counts, _ = np.histogram(alpaca_free_2[label], bins=bin_labels)
    print(f'Alpaca: T2: {counts}')
    counts, _ = np.histogram(alpaca_3[label], bins=bin_labels)
    print(f'Alpaca: T3: {counts}')
    counts, _ = np.histogram(alpaca_free_4[label], bins=bin_labels)
    print(f'Alpaca: T4: {counts}')
    print('##############')


toxic
....
Real Stories: [1047   46   17   16   12    8    6   11    9    2]
OPT-Line: [987  26  15   7   6   4  10   4   9   3]
OPT-256: [1075   27   22    8    8    9    3    2    7    6]
OPT-512: [1126   32   24   14   12    5    1   11    7   10]
LLaMA-Line: [987  26  15   7   6   4  10   4   9   3]
LLaMA-256: [1075   27   22    8    8    9    3    2    7    6]
LLaMA-512: [1126   32   24   14   12    5    1   11    7   10]
Alpaca: T1: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T2: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T3: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T4: [987  26  15   7   6   4  10   4   9   3]
##############


In [79]:
measures = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for m in measures:
    get_toxic_measure(m)

toxic
....
Real Stories: [1047   46   17   16   12    8    6   11    9    2]
OPT-Line: [987  26  15   7   6   4  10   4   9   3]
OPT-256: [1075   27   22    8    8    9    3    2    7    6]
OPT-512: [1126   32   24   14   12    5    1   11    7   10]
LLaMA-Line: [987  26  15   7   6   4  10   4   9   3]
LLaMA-256: [1075   27   22    8    8    9    3    2    7    6]
LLaMA-512: [1126   32   24   14   12    5    1   11    7   10]
Alpaca: T1: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T2: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T3: [987  26  15   7   6   4  10   4   9   3]
Alpaca: T4: [987  26  15   7   6   4  10   4   9   3]
##############
severe_toxic
....
Real Stories: [1174    0    0    0    0    0    0    0    0    0]
OPT-Line: [1071    0    0    0    0    0    0    0    0    0]
OPT-256: [1166    0    1    0    0    0    0    0    0    0]
OPT-512: [1242    0    0    0    0    0    0    0    0    0]
LLaMA-Line: [1071    0    0    0    0    0    0    0    0    0]
LLaMA-